# Data collection daily files
Gather public SPP Weis data from https://marketplace.spp.org/groups/operational-data-weis

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import duckdb
import polars as pl

import requests
from io import StringIO

# logging
import logging

In [ ]:
# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)


In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
os.chdir('../..')

In [ ]:
import src.data_collection as dc

## LMP settlement location prices

![_](../../imgs/lmp_settlement_location.PNG)

## LMP upsert

### LMP daily file

In [ ]:
# lookback 7 days to cover holidays and long weekends when data doesn't get updated
dc.collect_upsert_lmp(n_periods=7, daily_file=True)

In [ ]:
## to load specific day
# end_ts = pd.Timestamp('6/3/2023')
# dc.collect_upsert_lmp(end_ts=end_ts, n_periods=1, daily_file=True)